# 层和块

之前首次介绍神经网络时，我们关注的是具有单一输出的线性模型。
在这里，整个模型只有一个输出。
注意，单个神经网络
（1）接受一些输入；
（2）生成相应的标量输出；
（3）具有一组相关 *参数*（parameters），更新这些参数可以优化某目标函数。

然后，当考虑具有多个输出的网络时，
我们利用矢量化算法来描述整层神经元。
像单个神经元一样，层（1）接受一组输入，
（2）生成相应的输出，
（3）由一组可调整参数描述。
当我们使用softmax回归时，一个单层本身就是模型。
然而，即使我们随后引入了多层感知机，我们仍然可以认为该模型保留了上面所说的基本架构。

对于多层感知机而言，整个模型及其组成层都是这种架构。
整个模型接受原始输入（特征），生成输出（预测），
并包含一些参数（所有组成层的参数集合）。
同样，每个单独的层接收输入（由前一层提供），
生成输出（到下一层的输入），并且具有一组可调参数，
这些参数根据从下一层反向传播的信号进行更新。

事实证明，研究讨论“比单个层大”但“比整个模型小”的组件更有价值。
例如，在计算机视觉中广泛流行的ResNet-152架构就有数百层，
这些层是由*层组*（groups of layers）的重复模式组成。
这个ResNet架构赢得了2015年ImageNet和COCO计算机视觉比赛
的识别和检测任务 :cite:`He.Zhang.Ren.ea.2016`。
目前ResNet架构仍然是许多视觉任务的首选架构。
在其他的领域，如自然语言处理和语音，
层组以各种重复模式排列的类似架构现在也是普遍存在。

为了实现这些复杂的网络，我们引入了神经网络*块*的概念。
*块*（block）可以描述单个层、由多个层组成的组件或整个模型本身。
使用块进行抽象的一个好处是可以将一些块组合成更大的组件，
这一过程通常是递归的，如下图所示。
通过定义代码来按需生成任意复杂度的块，
我们可以通过简洁的代码实现复杂的神经网络。

![多个层被组合成块，形成更大的模型](../assets/blocks.svg)

从编程的角度来看，块由*类*（class）表示。
它的任何子类都必须定义一个将其输入转换为输出的前向传播函数，
并且必须存储任何必需的参数。
注意，有些块不需要任何参数。
最后，为了计算梯度，块必须具有反向传播函数。
在定义我们自己的块时，由于[自动微分](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/01.preliminaries/05.autograd.ipynb)
提供了一些后端实现，我们只需要考虑前向传播函数和必需的参数。

在构造自定义块之前，(**我们先回顾一下多层感知机**)
[多层感知机的简洁实现](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/03.multilayer-perceptrons/04.mlp_concise.ipynb)的代码。
下面的代码生成一个网络，其中包含一个具有256个单元和ReLU激活函数的全连接隐藏层，
然后是一个具有10个隐藏单元且不带激活函数的全连接输出层。

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
X, net(X)

(tensor([[0.9796, 0.8076, 0.5125, 0.5134, 0.9242, 0.8406, 0.8259, 0.5689, 0.1358,
          0.6521, 0.2873, 0.8786, 0.2026, 0.7159, 0.9612, 0.6912, 0.8726, 0.6182,
          0.8352, 0.2997],
         [0.8489, 0.2830, 0.0444, 0.2307, 0.4958, 0.4709, 0.3499, 0.5447, 0.8651,
          0.3285, 0.7093, 0.3332, 0.3095, 0.5998, 0.9309, 0.3219, 0.3583, 0.0834,
          0.6541, 0.5566]]),
 tensor([[ 0.3109, -0.0895,  0.0061,  0.0155,  0.1041,  0.2094, -0.0515, -0.3275,
          -0.1897,  0.1586],
         [ 0.2915, -0.0857,  0.0426,  0.0106,  0.0210,  0.1789,  0.0139, -0.1472,
          -0.2000,  0.1350]], grad_fn=<AddmmBackward0>))

在这个例子中，我们通过实例化`nn.Sequential`来构建我们的模型，
层的执行顺序是作为参数传递的。
简而言之，(**`nn.Sequential`定义了一种特殊的`Module`**)，
即在PyTorch中表示一个块的类，
它维护了一个由`Module`组成的有序列表。
注意，两个全连接层都是`Linear`类的实例，
`Linear`类本身就是`Module`的子类。
另外，到目前为止，我们一直在通过`net(X)`调用我们的模型来获得模型的输出。
这实际上是`net.__call__(X)`的简写。
这个前向传播函数非常简单：
它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。


[为什么net(X)就可以调用前向传播而不是net().forward(X)](https://kimi.moonshot.cn/share/cqkhq46sdv1fg0dab62g)

## [**自定义块**]

要想直观地了解块是如何工作的，最简单的方法就是自己实现一个。
在实现我们自定义块之前，我们简要总结一下每个块必须提供的基本功能。

1. 将输入数据作为其前向传播函数的参数。
1. 通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。
1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。
1. 存储和访问前向传播计算所需的参数。
1. 根据需要初始化模型参数。

在下面的代码片段中，我们从零开始编写一个块。
它包含一个多层感知机，其具有256个隐藏单元的隐藏层和一个10维输出层。
注意，下面的`MLP`类继承了表示块的类。
我们的实现只需要提供我们自己的构造函数（Python中的`__init__`函数）和前向传播函数。

In [2]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层

    # 定义模型的前向传播，即如何根据输入x返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

我们首先看一下前向传播函数，它以`X`作为输入，
计算带有激活函数的隐藏表示，并输出其未规范化的输出值。
在这个`MLP`实现中，两个层都是实例变量。
要了解这为什么是合理的，可以想象实例化两个多层感知机（`net1`和`net2`），
并根据不同的数据对它们进行训练。
当然，我们希望它们学到两种不同的模型。

接着我们[**实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层**]。
注意一些关键细节：
首先，我们定制的`__init__`函数通过`super().__init__()`
调用父类的`__init__`函数，
省去了重复编写模版代码的痛苦。
然后，我们实例化两个全连接层，
分别为`self.hidden`和`self.out`。
注意，除非我们实现一个新的运算符，
否则我们不必担心反向传播函数或参数初始化，
系统将自动生成这些。

我们来试一下这个函数：

In [3]:
net = MLP()
net(X)

tensor([[ 0.4527, -0.2148,  0.2249,  0.2976,  0.4442, -0.2574, -0.2377,  0.2058,
          0.0470, -0.2696],
        [ 0.2556, -0.0408,  0.1983,  0.1785,  0.3763, -0.2424, -0.2133,  0.2581,
          0.1177, -0.0239]], grad_fn=<AddmmBackward0>)

块的一个主要优点是它的多功能性。
我们可以子类化块以创建层（如全连接层的类）、
整个模型（如上面的`MLP`类）或具有中等复杂度的各种组件。
我们在接下来的章节中充分利用了这种多功能性，
比如在处理卷积神经网络时。

## [**顺序块**]

现在我们可以更仔细地看看`Sequential`类是如何工作的，
回想一下`Sequential`的设计是为了把其他模块串起来。
为了构建我们自己的简化的`MySequential`，
我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；
1. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的`MySequential`类提供了与默认`Sequential`类相同的功能。

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

`__init__`函数将每个模块逐个添加到有序字典`_modules`中。
读者可能会好奇为什么每个`Module`都有一个`_modules`属性？
以及为什么我们使用它而不是自己定义一个Python列表？
简而言之，`_modules`的主要优点是：
在模块的参数初始化过程中，
系统知道在`_modules`字典中查找需要初始化参数的子块。

当`MySequential`的前向传播函数被调用时，
每个添加的块都按照它们被添加的顺序执行。
现在可以使用我们的`MySequential`类重新实现多层感知机。

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0917,  0.1103,  0.2321,  0.0193, -0.1032, -0.1178,  0.0233,  0.0892,
         -0.0204, -0.4126],
        [-0.2346, -0.0174,  0.0833,  0.0238, -0.1547, -0.0375,  0.0396,  0.0048,
         -0.1052, -0.2233]], grad_fn=<AddmmBackward0>)

请注意，`MySequential`的用法与之前为`Sequential`类编写的代码相同
（如[多层感知机的简洁实现](https://github.com/lixinjie97/Deep_learning_tutorial/blob/main/03.multilayer-perceptrons/04.mlp_concise.ipynb)中所述）。

## [**在前向传播函数中执行代码**]

`Sequential`类使模型构造变得简单，
允许我们组合新的架构，而不必定义自己的类。
然而，并不是所有的架构都是简单的顺序架构。
当需要更强的灵活性时，我们需要定义自己的块。
例如，我们可能希望在前向传播函数中执行Python的控制流。
此外，我们可能希望执行任意的数学运算，
而不是简单地依赖预定义的神经网络层。

到目前为止，
我们网络中的所有操作都对网络的激活值及网络的参数起作用。
然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项，
我们称之为*常数参数*（constant parameter）。
例如，我们需要一个计算函数
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$的层，
其中$\mathbf{x}$是输入，
$\mathbf{w}$是参数，
$c$是某个在优化过程中没有更新的指定常量。
因此我们实现了一个`FixedHiddenMLP`类，如下所示：

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个`FixedHiddenMLP`模型中，我们实现了一个隐藏层，
其权重（`self.rand_weight`）在实例化时被随机初始化，之后为常量。
这个权重不是一个模型参数，因此它永远不会被反向传播更新。
然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情：
它运行了一个while循环，在$L_1$范数大于$1$的条件下，
将输出向量除以$2$，直到它满足条件为止。
最后，模型返回了`X`中所有项的和。
注意，此操作可能不会常用于在任何实际任务中，
我们只展示如何将任意代码集成到神经网络计算的流程中。

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(0.1800, grad_fn=<SumBackward0>)

我们可以[**混合搭配各种组合块的方法**]。
在下面的例子中，我们以一些想到的方法嵌套块。

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2225, grad_fn=<SumBackward0>)

## 效率

读者可能会开始担心操作效率的问题。
毕竟，我们在一个高性能的深度学习库中进行了大量的字典查找、
代码执行和许多其他的Python代码。
Python的问题[全局解释器锁](https://wiki.python.org/moin/GlobalInterpreterLock)
是众所周知的。
在深度学习环境中，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。

## 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的顺序连接由`Sequential`块处理。
* [全连接层图解，每两个节点的线基本可以认为是两点之间的权重](https://blog.csdn.net/Y_hero/article/details/88296967)

## 练习

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
1. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。
1. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

### 练习一

1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？

&emsp;&emsp;如果将MySequential中存储块的方式从OrderedDict更改为Python列表，代码可以正常计算，但并没有注册给Module。无法像`_modules`一样使用很多内置方法完成已经实现的功能。如无法通过`net.state_dict()`访问模型的网络结构和参数。

In [9]:
import torch.nn as nn
import torch

class MySequential(nn.Module):
    # 使用OrderedDict存储块的类
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

class MySequential_list(nn.Module):
    # 使用list存储块的类
    def __init__(self, *args):
        super(MySequential_list, self).__init__()
        self.sequential = []
        for module in args:
            self.sequential.append(module)

    def forward(self, X):
        for module in self.sequential:
            X = module(X)
        return X

X = torch.rand(1, 10)
# 初始化两个block，确保传给MySequential和MySequential_list的是一样的参数。消除nn.Linear初始化时生成随机参数对结果的影响
block1 = nn.Linear(10, 20)
block2 = nn.Linear(20, 10)
net = MySequential(block1, nn.ReLU(), block2)
net_list = MySequential_list(block1, nn.ReLU(), block2)

&emsp;&emsp;对比两种方式的结果，可以发现输出完全一致。

In [10]:
net_y = net(X)
net_list_y = net_list(X)

print("采用python OrderedDict的方式存储：", net_y, sep='\n')
print("\n")
print("采用python列表的方式存储：", net_list_y, sep='\n')
print("\n")
print("两种方式的计算结果是否一致：", net_y.equal(net_list_y))

采用python OrderedDict的方式存储：
tensor([[ 0.3188, -0.4795,  0.0580,  0.0034, -0.4343,  0.1757, -0.1644, -0.0972,
         -0.0207,  0.1600]], grad_fn=<AddmmBackward0>)


采用python列表的方式存储：
tensor([[ 0.3188, -0.4795,  0.0580,  0.0034, -0.4343,  0.1757, -0.1644, -0.0972,
         -0.0207,  0.1600]], grad_fn=<AddmmBackward0>)


两种方式的计算结果是否一致： True


&emsp;&emsp;但是在查看模型结构和参数时存在差异。

In [11]:
# 使用_modules方便打印net的网络结构和参数，而list则无法做到
print("采用python OrderedDict的方式存储：\n")
print("网络结构：\n", net)
# 为了可视化效果简洁，这里只展示了每个参数的部分值
print("参数：", "\n".join([str(k) + str("") + str(v[:2]) for k, v in net.state_dict().items()]), sep='\n')
print("\n")
print("采用python列表的方式存储：\n")
print("网络结构：\n", net_list)
print("参数：", net_list.state_dict())

采用python OrderedDict的方式存储：

网络结构：
 MySequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
)
参数：
0.weighttensor([[ 0.2062,  0.2842, -0.0201, -0.1943,  0.0322,  0.1790,  0.2235, -0.0422,
          0.1049, -0.1508],
        [-0.2228, -0.2805,  0.2787, -0.0207, -0.1200, -0.2629, -0.0936,  0.0953,
          0.1250, -0.1247]])
0.biastensor([0.1850, 0.1082])
2.weighttensor([[ 0.0085,  0.1576,  0.0462,  0.0420,  0.0041, -0.0271, -0.1531,  0.1537,
         -0.1524,  0.2050, -0.0027, -0.1979,  0.1770, -0.1843, -0.2013,  0.0358,
         -0.0762,  0.0999, -0.1618, -0.0577],
        [-0.1257, -0.0228, -0.2214, -0.1463, -0.1593,  0.0114, -0.1784, -0.0089,
         -0.1489, -0.0872,  0.2198,  0.0937,  0.0143, -0.1471,  0.1375,  0.0050,
          0.0439, -0.1446, -0.1936,  0.1032]])
2.biastensor([ 0.0776, -0.1245])


采用python列表的方式存储：

网络结构：
 MySequential_list()
参数： OrderedDict()


### 练习二

2. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。

&emsp;&emsp;在本书7.4节中`GoogleLet`模型中的`Inception`块使用了并行技术。下面代码实现了一个并行网络，有两个子网络组成。输入数据先分别经过两个子网络的计算，分别得到两个部分的输出结果，然后在通道维度上合并结果得到最终输出。

&emsp;&emsp;其中，`net1`和`net2`分别表示两个子网络，`torch.cat`在指定维度上拼接张量。输出结果的大小为`torch.Size([2, 36])`，其中第一个维度表示batch_size为2，第二个维度表示输出特征图的通道数为36，正好为两个模型的输出特征图通道数之和（12+24=36）。

In [12]:
import torch.nn as nn
import torch

class Parallel(nn.Module):
    # 自定义并行块
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1 # 第一个子网络
        self.net2 = net2 # 第二个子网络

    def forward(self, X):
        x1 = self.net1(X) # 第一个子网络的输出
        x2 = self.net2(X) # 第二个子网络的输出
        return torch.cat((x1, x2), dim=1) # 在通道维度上合并输出结果

X = torch.rand(2, 10) # 输入数据
net = Parallel(nn.Sequential(nn.Linear(10, 12), nn.ReLU()), nn.Sequential(nn.Linear(10, 24), nn.ReLU())) # 实例化并行网络
output = net(X)
print("模型结构：", net, sep="\n") # 查看模型结构
print("模型输出的形状：", output.size()) # 输出结果的大小

模型结构：
Parallel(
  (net1): Sequential(
    (0): Linear(in_features=10, out_features=12, bias=True)
    (1): ReLU()
  )
  (net2): Sequential(
    (0): Linear(in_features=10, out_features=24, bias=True)
    (1): ReLU()
  )
)
模型输出的形状： torch.Size([2, 36])


### 练习三

3. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。

&emsp;&emsp;下面代码定义了一个函数`create_network`，该函数接受四个参数`num_instances`、`input_size`、`hidden_size`和`output_size`，并返回一个Sequential模型。

&emsp;&emsp;其中，该网络模型首先包含`num_instances`个相同的线性层，每个线性层有两个子层：一个输入维度为`input_size`，输出维度为`hidden_size`的全连接层，和一个ReLU非线性激活层。然后，这`hidden_size`个线性层连接在一起作为整个网络的前馈部分。最后，额外添加一个输出层，其输入维度为`input_size`，输出维度为`output_size`。

&emsp;&emsp;因此，最终的网络结构是有`output_size`个相同的线性层组成的前馈神经网络，每个线性层内部包含一个全连接层和ReLU激活层，以及一个独立的输出层。

In [13]:
import torch.nn as nn
import torch

def create_network(num_instances, input_size, hidden_size, output_size):
    # 创建一个线性层
    linear_layer = nn.Sequential(
        nn.Linear(input_size, hidden_size), nn.ReLU(),
        nn.Linear(hidden_size, input_size)
    )

    # 创建多个相同结构的实例并连接
    instances = [linear_layer for _ in range(num_instances)]
    network = nn.Sequential(*instances)

    # 添加输出层
    output_layer = nn.Linear(input_size, output_size)
    network.add_module("output", output_layer)

    return network

# 模型参数
num_instances = 3
input_size = 10
hidden_size = 5
output_size = 2

# 示例用法
net = create_network(num_instances=num_instances, input_size=input_size, 
                    hidden_size=hidden_size, output_size=output_size) # 重复3次相同的线性层
# 查看模型结构
print(net)

Sequential(
  (0): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (output): Linear(in_features=10, out_features=2, bias=True)
)
